# import def func

In [1]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [2]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20):
    col_src = [
        "name", "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
def get_tdx_all_from_h5(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

## init data

### get_tdx_all

In [3]:
h5_day=get_tdx_all_from_h5(resample='d')

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


### sina

### sina_all

In [4]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-19',
 code
 301013    利和兴
 Name: name, dtype: object)

In [5]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/20/25 16:25:39'

code
600302   2025-06-20 15:00:01
002167   2025-06-20 15:00:00
Name: ticktime, dtype: datetime64[ns]

85.6649484316508

#### 实时成交

In [6]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(2767032, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-20 14:58:58,66.9,67.88,66.75,67.55,16223684,67.11
2025-06-20 14:59:40,66.9,67.88,66.75,67.55,16223684,67.18
2025-06-20 15:00:01,66.9,67.88,66.75,67.55,16398684,67.07


1.0

总量:8715.97 亿


ticktime    2025-06-21 00:00:00
close                  67.21773
volume                     0.13
vol                        16.4
amount                    11.02
Name: 603019, dtype: object

'2025-06-20'

#### 读取实时成交个股

In [7]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5420, 5),
         volume   name
 code                 
 300308    0.73   中际旭创
 300468    0.71   四方精创
 600519    0.57   贵州茅台
 300476    0.55   胜宏科技
 300502    0.54    新易盛
 601318    0.53   中国平安
 300773    0.49    拉卡拉
 002475    0.45   立讯精密
 002463    0.45   沪电股份
 002104    0.43   恒宝股份
 000858    0.41  五 粮 液
 300059    0.39   东方财富
 300750    0.38   宁德时代
 003040    0.36    楚天龙
 002384    0.35   东山精密
 002017    0.33   东信和平
 603300    0.32   海南华铁
 300255    0.31   常山药业
 002050    0.31   三花智控
 002594    0.29    比亚迪)

##### 合并实时成交量及SinaDM

In [8]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey

In [9]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [63]:
filterIDX=2

### resample 3d

In [64]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d'}
filter_keys=list(filter_query.keys())

In [82]:
h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

In [83]:
tdx_3d.loc['000002'].close

6.29

In [67]:
tdx_3d.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
tdx_select = tdx_3d.query(f'{get_filter(filterIDX)}')

show_tdx_data(tdx_select,limit=10)

(5188, 218)

3周期新高大于Ma5: lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d
Count:192 218


name        date  close   hmax   lmin  perc1d  nclose   llow  perc3d  \
code                                                                           
600156   NaN  2025-04-08    NaN   7.00   4.05    1.20     NaN   4.42    22.5   
000159  国际实业  2025-04-08   6.54   5.68   4.21   34.31    6.61   4.50     4.1   
000408  藏格矿业  2025-04-08  40.81  38.28  31.00    0.10   40.78  31.20    22.3   
000554  泰山石油  2025-04-08   7.52   6.46   5.00   29.41    7.68   5.24    18.0   
000682  东方电子  2025-04-08  10.21  10.22   8.69    1.20   10.24   8.82    22.3   
000697  ST炼石  2025-04-29   7.09   8.01   6.10    1.30    7.16   6.19     0.0   
000731  四川美丰  2025-04-30   6.86   7.19   6.41    1.30    6.88   6.53     0.1   
000852  石化机械  2025-04-30   6.64   8.10   5.53    0.00    6.71   5.95    20.3   
000968  蓝焰控股  2025-04-08   8.10   6.73   5.62   48.81    8.20   5.84    20.3   
001314  亿道信息  2025-04-08  45.31  49.59  34.59    1.20   46.44  37.81     1.3   

        perc5d  
code            
600156    14.3  
000159    12.3  
000408     1.3  
000554     1.3  
000682     1.3  
000697    27.3  
000731     1.3  
000852     1.2  
000968     1.3  
001314     1.3

limit_output extension: Maximum message size of 2000 exceeded with 3390 characters

In [68]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
tdx_3d.shape
t3d_code = tdx_3d.query(f'{get_filter(1)}')
show_tdx_data(t3d_code)

(5188, 218)

3周期新高上轨: lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3
Count:80 218


name        date  close   hmax   lmin  perc1d  nclose   llow  perc3d  \
code                                                                            
600156    NaN  2025-04-08    NaN   7.00   4.05    1.20     NaN   4.42   22.50   
000504  *ST生物  2025-04-30   8.98  10.00   5.43    1.30    8.90   5.46    1.20   
000554   泰山石油  2025-04-08   7.52   6.46   5.00   29.41    7.68   5.24   18.00   
001389   广合科技  2025-04-08  56.09  58.38  37.38    0.00   57.19  40.50    0.10   
002015   协鑫能科  2025-04-08  10.68   8.70   6.34   31.09   10.86   6.67   16.09   
002017   东信和平  2025-04-08  15.64  11.53   8.07   57.41   15.61   8.24    0.00   
002104   恒宝股份  2025-03-28  14.70  11.06   6.42   33.09   14.62   6.71   14.00   
002130   沃尔核材  2025-04-08  22.86  21.80  14.80    0.00   22.98  16.05    0.00   
002199  *ST东晶  2025-04-08   9.04   7.61   4.50   28.20    8.80   4.74   22.20   
002207   准油股份  2025-04-08   9.62   6.11   4.42   55.41   10.20   4.68   19.80   
002235   安妮股份  2025-04-08   9.03   5.72   4.12   37.31    8.88   4.33   26.09   
002366   融发核电  2025-04-08   7.49   8.24   4.10   22.80    7.72   4.34   23.70   
002463   沪电股份  2025-04-08  43.22  36.88  23.08   35.41   42.97  24.44   17.00   
002558   巨人网络  2025-04-08  21.19  16.16  11.49    1.20   20.91  11.94   25.70   
002741   光华科技  2025-04-08  19.41  17.55  12.47    1.30   19.41  13.03    0.10   
002826   易明医药  2025-04-08  24.70  19.61   8.18    1.30   24.23   9.03   28.80   
002898  *ST赛隆  2025-04-29  12.66  12.99   5.77    1.30   12.66   5.77   10.00   
002916   深南电路  2025-04-08  99.25  97.00  75.00   32.41   99.38  76.19    1.20   
002940    昂利康  2025-04-08  31.94  23.73  11.44    1.30   31.53  12.12   18.30   
003040    楚天龙  2025-04-08  23.33  17.45  10.23   45.09   21.95  10.71   -1.10   

        perc5d  
code            
600156   14.30  
000504   12.30  
000554    1.30  
001389

limit_output extension: Maximum message size of 2000 exceeded with 5956 characters

### resample W

In [84]:
h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w
tdx_w = cct.combine_dataFrame(h5_w,sinadm)

In [70]:
tdx_w.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select_w = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# show_tdx_data(tdx_select_w)

(5186, 220)

In [71]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
tw_code = tdx_w.query(f'{get_filter(filterIDX)}')

show_tdx_data(tw_code,limit=10)

3周期新高大于Ma5: lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d
Count:651 220


name        date  close   hmax   lmin  perc1d  nclose   llow  perc3d  \
code                                                                            
300959    NaN  2025-04-11    NaN  52.47  30.17    1.30     NaN  30.17     5.3   
600156    NaN  2025-01-10    NaN   6.39   4.05    1.20     NaN   4.31    21.7   
000001   平安银行  2025-04-11  11.84  11.61  10.12    1.20   11.80  10.12     1.3   
000020   深华发Ａ  2025-04-11  13.65  14.71   9.22   44.91   14.01   9.22     1.1   
000049   德赛电池  2025-04-11  21.58  24.81  17.91    0.00   21.72  17.91     0.0   
000159   国际实业  2025-04-11   6.54   7.67   4.21   39.41    6.61   4.21     4.1   
000407   胜利股份  2025-04-11   3.58   3.72   2.76   28.91    3.58   2.76     0.1   
000408   藏格矿业  2024-12-20  40.81  36.81  26.75    1.20   40.78  26.75     0.0   
000428   华天酒店  2025-01-10   3.21   3.94   2.73    0.00    3.24   2.90     1.3   
000430  ST张家界  2025-01-03   7.92   9.17   5.56   27.30    7.97   5.80     1.2   

        perc5d  
code            
300959    -0.9  
600156     1.2  
000001     1.2  
000020     1.2  
000049     0.0  
000159    -0.1  
000407    -0.1  
000408     0.0  
000428    -0.1  
000430     0.1

limit_output extension: Maximum message size of 2000 exceeded with 3382 characters

### resample M

In [85]:
h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
tdx_m = cct.combine_dataFrame(h5_m,sinadm)

In [86]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp')
tm_code = tdx_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code)

3周期新高大于Ma5: lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d
Count:946 218


name        date  close     hmax     lmin  perc1d  nclose     llow  \
code                                                                         
999999   NaN  2025-01-27    NaN  3352.00  2690.00    0.10     NaN  3140.00   
601028   NaN  2025-02-28    NaN    14.58    10.52    1.10     NaN    11.14   
600156   NaN  2024-09-30    NaN     5.48     2.71   54.31     NaN     3.24   
000001  平安银行  2025-04-30  11.84    11.60     9.00    1.30   11.80    10.12   
000010  美丽生态  2024-09-30   2.90     3.03     1.38    1.20    2.91     1.64   
000020  深华发Ａ  2025-04-30  13.65    15.72     9.00    1.30   14.01     9.22   
000059  华锦股份  2025-01-27   5.08     5.37     3.58    1.10    5.14     4.28   
000159  国际实业  2025-04-30   6.54     7.44     4.83    1.30    6.61     4.21   
000407  胜利股份  2024-09-30   3.58     3.45     2.47   35.81    3.58     2.60   
000408  藏格矿业  2024-12-31  40.81    35.84    21.14   36.09   40.78    26.75   
000426  兴业银锡  2024-12-31  15.04    14.21     9.50   37.59   15.15    11.10   
000513  丽珠集团  2025-02-28  35.88    39.56    32.97    1.30   36.06    32.97   
000519  中兵红箭  2025-01-27  17.66    16.97    11.14    1.10   17.88    13.18   
000520  凤凰航运  2024-09-30   5.35     6.41     2.00    1.10    5.43     2.00   
000533  顺钠股份  2024-09-30   6.65     6.55     2.98    1.20    6.62     3.07   
000546  金圆股份  2025-04-30   4.68     6.34     3.73    1.20    4.81     3.67   
000553  安道麦A  2024-09-30   6.97     7.37     4.10    1.20    7.04     4.10   
000555  神州信息  2025-04-30  12.31    13.48     7.87    1.20   12.26     9.33   
000561  烽火电子  2025-04-30   8.53    10.93     5.62    0.10    8.62     6.58   
000576  甘化科工  2024-09-30   8.39     7.55     5.21    1.10    8.44     5.21   

        perc3d  perc5d  
code                    
999999   -0.10    1.20  
601028    1.10    0.00  
600156    0.00   37.50  
000001   -1.10    1.20  
000010   -1.10    1.10

limit_output extension: Maximum message size of 2000 exceeded with 5902 characters

### resample D

In [88]:
h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
tdx_d = cct.combine_dataFrame(h5_d,sinadm)

In [89]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
td_code = tdx_d.query(f'{get_filter(filterIDX)}')
show_tdx_data(td_code,limit=10)
if len(td_code) < 30:
    write_code_to_blk(td_code.index,'065',rewrite=True)

3周期新高大于Ma5: lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d
Count:206 221


name        date  close   hmax   lmin  perc1d  nclose   llow  perc3d  \
code                                                                          
600156  NaN  2025-04-08    NaN   7.50   5.14    1.20     NaN   4.42    0.00   
001277  NaN  2025-04-08    NaN  45.44  37.00    1.30     NaN  31.19    1.30   
002141  NaN  2025-04-08    NaN   4.07   2.30   33.19     NaN   1.56    0.00   
002200  NaN  2025-04-08    NaN   9.03   7.67    1.30     NaN   5.13   -1.10   
002490  NaN  2025-03-28    NaN   5.27   3.90   27.30     NaN   3.07   17.41   
002650  NaN  2025-04-08    NaN   8.07   5.88    1.30     NaN   4.47   -1.10   
002700  NaN  2025-04-07    NaN   9.18   8.43    0.00     NaN   8.12    1.20   
002742  NaN  2025-04-07    NaN   4.87   3.79    0.00     NaN   3.54   30.59   
002951  NaN  2025-04-07    NaN  14.88   7.84    1.30     NaN   6.79    5.30   
300029  NaN  2025-04-07    NaN   5.61   4.74    1.20     NaN   4.19    0.00   

        perc5d  
code            
600156   33.41  
001277    0.00  
002141   11.90  
002200    1.30  
002490    0.10  
002650    0.20  
002700    0.10  
002742   12.20  
002951   -0.90  
300029    0.10

limit_output extension: Maximum message size of 2000 exceeded with 3361 characters

In [90]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
td_code2 = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
show_tdx_data(td_code2,limit=10)


Count:40 221


name        date  close   hmax   lmin  perc1d  nclose   llow  perc3d  \
code                                                                           
002080  中材科技  2025-04-08  17.58  17.52  14.03    1.30   17.58  12.58     1.3   
002235  安妮股份  2025-04-08   9.03   7.12   5.17    1.30    8.88   4.33     0.0   
002374  中锐股份  2025-04-08   3.11   2.83   2.61    1.30    3.05   2.12     0.2   
002558  巨人网络  2025-04-08  21.19  20.92  13.39    1.30   20.91  11.94    -0.9   
002676  顺威股份  2025-04-17   6.39   6.05   5.54    1.30    6.30   5.51     0.0   
002685  华东重机  2025-04-08   7.36   8.17   6.52    1.30    7.12   4.97     0.3   
002807  江阴银行  2025-04-07   4.92   4.75   4.16    1.30    4.92   3.88     1.3   
002826  易明医药  2025-04-08  24.70  23.73  10.26    1.20   24.23   9.03    27.3   
003040   楚天龙  2025-04-08  23.33  19.20  12.37   47.81   21.95  10.71     0.3   
300269  联建光电  2025-04-07   6.17   4.18   3.83   52.09    6.01   2.95     1.2   

        perc5d  
code            
002080    -1.1  
002235    17.3  
002374    -1.1  
002558    16.2  
002676    -1.1  
002685    -0.1  
002807     1.2  
002826    -0.9  
003040    -1.1  
300269    -1.1

limit_output extension: Maximum message size of 2000 exceeded with 3293 characters

#### 多周期筛选

In [92]:
# '002476' in td_code

False

In [93]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))

if len(code_f) > 0:
    show_tdx_data(tw_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:946
tw_code:651
t3d_code:80
td_code:206
Count:13 220


name        date  close   hmax   lmin  perc1d  nclose   llow  perc3d  \
code                                                                           
600156   NaN  2025-01-10    NaN   6.39   4.05    1.20     NaN   4.31   21.70   
002558  巨人网络  2025-01-10  21.19  16.67  10.97   52.00   20.91  10.97    7.30   
300157  新锦动力  2025-04-25   5.44   4.01   2.28   66.88    5.65   2.66    4.10   
002235  安妮股份  2025-04-11   9.03   7.29   4.12   34.31    8.88   4.12    1.20   
688619   罗普特  2025-01-10  12.12  10.25   7.10    1.10   12.36   7.80    1.30   
002826  易明医药  2025-01-10  24.70  12.71   8.18   51.91   24.23   9.55   40.41   
301357  北方长龙  2025-01-27  60.00  39.66  26.45   87.69   62.38  28.48    0.00   
872895  花溪科技  2025-01-27  52.53  30.20  11.71    1.10   48.81  11.91    1.20   
839719  宁新新材  2025-01-27  23.70  15.48   8.75   55.50   22.64   8.81    0.00   
003040   楚天龙  2025-04-11  23.33  16.64  10.23   42.81   21.95  10.23   22.50   

        perc5d  
code            
600156    1.20  
002558    0.00  
300157   -0.90  
002235   -0.10  
688619   -0.90  
002826    1.10  
301357    1.30  
872895    1.20  
839719   40.59  
003040   -0.10

limit_output extension: Maximum message size of 2000 exceeded with 3369 characters

#### check data

In [79]:
code_check = [co for co in code_f if co in tdx_3d.index]
len(code_check)

13

In [80]:
show_tdx_data(td_code.loc[code_f])

Count:13 221


name        date  close   hmax   lmin  perc1d  nclose   llow  perc3d  \
code                                                                           
600156   NaN  2025-04-08    NaN   7.50   5.14    1.20     NaN   4.42    0.00   
002558  巨人网络  2025-04-08  21.19  20.92  13.39    1.30   20.91  11.94   -0.90   
300157  新锦动力  2025-04-07   5.44   5.07   3.55    0.00    5.65   2.42   38.91   
002235  安妮股份  2025-04-08   9.03   7.12   5.17    1.30    8.88   4.33    0.00   
688619   罗普特  2025-04-07  12.12  12.43   9.60    1.10   12.36   7.10   -0.90   
002826  易明医药  2025-04-08  24.70  23.73  10.26    1.20   24.23   9.03   27.30   
301357  北方长龙  2025-04-07  60.00  45.00  30.70   62.00   62.38  26.45    0.00   
872895  花溪科技  2025-04-07  52.53  43.47  18.39    1.20   48.81  13.18    0.30   
839719  宁新新材  2025-04-07  23.70  17.45  11.23    1.20   22.64   8.90    0.00   
003040   楚天龙  2025-04-08  23.33  19.20  12.37   47.81   21.95  10.71    0.30   
002207  准油股份  2025-04-08   9.62   7.55   5.69   30.30   10.20   4.68   17.41   
002017  东信和平  2025-04-08  15.64  13.27   9.54   42.09   15.61   8.24   17.30   
300940   南极光  2025-04-07  24.20  20.91  16.14    1.20   24.39  12.55    5.30   

        perc5d  
code            
600156   33.41  
002558   16.20  
300157   25.41  
002235   17.30  
688619    1.30  
002826   -0.90  
301357   27.30  
872895   -1.00  
839719    1.20  
003040   -1.10  
002207   15.20  
002017    1.20  
300940    8.80

limit_output extension: Maximum message size of 2000 exceeded with 4076 characters

### Write toblk

In [81]:
# write_code_to_blk(code_f,'077',rewrite=False)